In [1]:
import pymongo
import os,csv,sys
import openslide
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers
import pprint
client = pymongo.MongoClient('localhost',27017)
dsa_load_errors_db = client['DSA_LoadErrors']

In [ ]:
reload(dsa)
reload(cdsa_helpers)

In [3]:
## This is specific to a given fle system and or structurme
slide_root = '/bigdata/PanCan_Images/'  ##Base Path for Slides
dsa_slide_db = client['PanCanDSA_Slide_Data_N15']  ### These need to be configured for the specific project

feature_db = client['N15PanCan_BoundsOnly']

### To generalize this, need to describe organization, most common will be  PATIENT/STAIN_TYPE as subdirectories
subj_dir_list = [x for x in os.listdir(slide_root) if os.path.isdir(os.path.join(slide_root,x))]
print len(subj_dir_list),"Potential Patient directories were identified"
print subj_dir_list

5 Potential Patient directories were identified
['GBM', 'LGG', 'LUAD', 'SARC', 'SKCM']


In [ ]:
def find_rawslide_lists( slide_root_path ):
        """project_name is passed along with the potentially more than one root image path for ndpi files"""
        slide_files = []

        slide_root_path  = slide_root_path.rstrip('/')
        print slide_root_path
        for dpath, dnames, fnames in os.walk( slide_root_path, followlinks=True):
                
                for file in fnames:
                    if '.ndpi' in file or '.svs' in file:
                                slide_files.append(dpath +'/'+file)
        print len(slide_files),"SVS or NDPI files were located"
        return slide_files

In [ ]:
print subj_dir_list
print dsa_slide_db['RawSlideData'].count()

In [ ]:
### So this creates a document WITHIN the current database to store raw information about the slides
## Now that I think about this, I really should not store the filename but the File Hash as I.. want to change the filenames

for sd in subj_dir_list:
    print sd,"is being processed",
    curr_svs_slide_list = find_rawslide_lists(  os.path.join(slide_root,sd)  )
    slides_processed = newly_processed = dup_slide = rescanned_slides =  0
    for sld in curr_svs_slide_list:

        slide_name = os.path.basename(sld)
        qry = dsa_slide_db['RawSlideData'].find_one( {'slide_name':slide_name})
        #print qry
        if not qry:
            fs = os.path.getsize(sld)
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'ndpi', client)
            if openslide_could_open:
                prep_type = 'Unknown'
                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                dsa_slide_db['RawSlideData'].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                ###Need to flag/load this in to an error database

        else:
            fs = os.path.getsize(sld)
            ## Double check if file size matches
#             if qry['file_size'] != fs:
#                 #print "File size mismatch??",fs,qry['file_size'],qry['slide_w_path'],sld
#                 load_errors_db['rescanned_slides'].insert_one( {'loaded_slide': qry['slide_w_path'], 'rescanned_slide': sld}             )
#                 rescanned_slides +=1 
#             else:
#                 dup_slide +=1

    
        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


In [ ]:
def getFeatureInfo( slide_name, group_name):
    """Queries the feature Database and determine if there are any segmentations loaded"""
    print "received %s and %s" % ( slide_name, group_name)
# print dsa_slide_db['RawSlideData'].count()
# dsa_slide_db['RawSlideData'].delete_many({})
# print dsa_slide_db['RawSlideData'].count()


In [ ]:
### I now want to reformat all of this data to make it more useful for DSA ...

In [14]:
all_slides = dsa_slide_db['RawSlideData'].find()

## Since this is pretty easy to recreate, I'm going to empty the current database

dsa_slide_db['PanCanDSA_Slide_Data'].delete_many({})

for s in all_slides:
    slide_dict = {}
    #pt_id = s['slide_w_path'].split('/')[-1]
    slideGroup = s['slide_w_path'].split('/')[-2]
    pt_id = s['slide_w_path'].split('/')[-1].split('.')[0]
   # stain_type = s['slide_w_path'].split('/')[-2]
    stain_type = 'UNK'
    slide_dict = s
    slide_dict.pop('_id',None)
    
    slide_dict['pt_id'] = pt_id
    slide_dict['stain_type'] = stain_type
    ### Obfuscating the global file path so everything is relative to some base path for the archive/
    slide_dict['thumbnail_image'] = '/thumbnail/' + s['slide_w_path'].replace(slide_root,'')
    slide_dict['slide_w_path'] = '/DZIMS/' + s['slide_w_path'].replace(slide_root,'')+'.dzi'
#    slide_dict['slide_w_path'] = '/DZIMS' + s['slide_w_path']+'.dzi'
    slide_dict['slideGroup'] = slideGroup
    
    slide_dict['HasPathReport'] = True
    slide_dict['PathReportURL'] = "TBD"
  
    
    slide_dict['TumorType'] = slideGroup
    
    
    slide_name_noext = s['slide_name'].replace('.svs','')
    slide_dict['slide_name_noext'] = slide_name_noext
    slide_dict['slide_nouid'] = pt_id
    
    
    FeatureColl = "Features.V1.%s.%s" % ( slideGroup, slide_name_noext)
    FeatObjs = feature_db[FeatureColl].count()
    ##Feature database should be  Features.V1.[TumorType].SlideName (without the trailing crap)
    slide_dict['FeatureDB_CollName'] =FeatureColl
    if FeatObjs > 0:
        slide_dict['HasAnnotations'] = True
    else:
        slide_dict['HasAnnotations'] = False
        
    slide_dict['FeatObjs'] = FeatObjs
    
    dsa_slide_db['PanCanDSA_Slide_Data'].insert_one(slide_dict)
    
 

In [15]:
coll_list = dsa_slide_db['PanCanDSA_Slide_Data'].distinct('slideGroup')
print coll_list

[u'GBM', u'LGG', u'LUAD', u'SARC', u'SKCM']


In [10]:
#all_slides = dsa_slide_db['RawSlideData'].find()
print feature_db.collection_names()

[u'Features.V1.SARC.TCGA-DX-AATS-01Z-00-DX4', u'Features.V1.SARC.TCGA-DX-A6Z2-01A-01-TSA', u'Features.V1.SARC.TCGA-SI-AA8C-01Z-00-DX5', u'Features.V1.LGG.TCGA-HT-7879-01Z-00-DX2.1F7DF69E-64B9-4B29-B4EE-FDE88E51E1F2', u'Features.V1.SARC.TCGA-DX-A7EU-01Z-00-DX4', u'Features.V1.LGG.TCGA-TM-A84F-01Z-00-DX2', u'Features.V1.SKCM.TCGA-BF-AAP7-01Z-00-DX1', u'Features.V1.GBM.TCGA-02-0003-01Z-00-DX1.6171b175-0972-4e84-9997-2f1ce75f4407', u'Features.V1.SARC.TCGA-DX-A8BJ-01Z-00-DX2', u'Features.V1.LGG.TCGA-DU-7006-01Z-00-DX1', u'Features.V1.SARC.TCGA-3B-A9HL-01Z-00-DX1', u'Features.V1.SARC.TCGA-DX-A48O-01Z-00-DX1', u'Features.V1.SKCM.TCGA-GN-A264-01Z-00-DX1', u'Features.V1.SARC.TCGA-DX-A2IZ-01Z-00-DX1', u'Features.V1.SARC.TCGA-SI-A71Q-01Z-00-DX4', u'Features.V1.LGG.TCGA-HT-7607-01Z-00-DX4', u'Features.V1.GBM.TCGA-06-0187-01Z-00-DX2.87f7b94d-c9d3-4ed3-9bb3-2e37d810e2ea', u'Features.V1.GBM.TCGA-12-0769-01Z-00-DX7.fee8ea19-a415-451d-8a37-91de4e0ce748', u'Features.V1.GBM.TCGA-12-0620-01Z-00-DX3.e12d45

In [ ]:


print dsa_slide_db['PanCanDSA_Slide_Data'].find_one({'slide_name': {"$regex": "TCGA-3B-A9I1-01Z-00-DX1"}  }
                                             
                                             )


#{ <field>: { $regex: /pattern/, $options: '<options>' } }

In [ ]:
dsa_slide_db['PanCanDSA_Slide_Data'].count()

In [ ]:
print dsa_slide_db['DSA_Slide_Data'].distinct('pt_id')

In [ ]:
cur= dsa_slide_db['DSA_Slide_Data'].find({'pt_id':'ADRC50-10'})
for c in cur:
    print c

In [ ]:

## Going to create a cleanup and /or reformatted collection for the DSA Viewer
for s in dsa_slide_db['ADRC'].find():
    keys_of_interest = ['width','height']
    print s['slide_w_path']
    print s.keys()
    sys.exit()
###
